In [ ]:
!git clone https://github.com/facebookresearch/sam3.git
%cd sam3
!pip install -e .
!pip install matplotlib huggingface_hub decord einops hydra-core omegaconf timm submitit iopath pandas pycocotools

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import requests
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import io

# 1. Download image from provided URL
url = "https://thumbs.dreamstime.com/b/nyc-intersection-crowded-busy-people-cars-yellow-taxis-iconic-traffic-street-business-manhattan-new-york-city-114460105.jpg"
print(f"📥 Downloading New York image...")
response = requests.get(url)
image = Image.open(io.BytesIO(response.content)).convert("RGB")

# 2. Configure image in processor (Assuming model/processor is already loaded)
inference_state = processor.set_image(image)

# 3. Challenging Prompts
# - "person": Crowd counting test
# - "yellow taxi": Compound concept test (Adjective + Noun)
# - "traffic light": Small object test
prompts = ["person", "yellow taxi", "traffic light"]

# Visual configuration
plt.figure(figsize=(16, 10))
plt.imshow(image)
plt.axis('off')
ax = plt.gca()

# Manual colors for readability (R, G, B, Alpha)
class_colors = [
    [1, 0, 0, 0.4],    # Red (People)
    [1, 1, 0, 0.4],    # Yellow (Taxis)
    [0, 1, 0, 0.4]     # Green (Traffic lights)
]

print("🧠 Processing concepts...")

total_objects = 0

for i, prompt in enumerate(prompts):
    output = processor.set_text_prompt(
        state=inference_state, 
        prompt=prompt
    )
    
    class_count = 0
    color = np.array(class_colors[i])
    
    for mask, score in zip(output["masks"], output["scores"]):
        # Confidence > 0.35 to avoid crowd noise
        if score > 0.35:
            h, w = mask.shape[-2:]
            mask_data = mask.reshape(h, w, 1).cpu().numpy()
            
            # Apply color to mask
            colored_img = mask_data * color.reshape(1, 1, -1)
            ax.imshow(colored_img)
            
            # (Optional) Draw simple box if desired
            # show_box(output["boxes"][...]) 
            
            class_count += 1
            total_objects += 1
            
    print(f"   -> Found {class_count} occurrences of '{prompt}'")

plt.title(f"SAM 3 'In the Wild' Test | Total Detected: {total_objects}")
plt.show()